In [1]:
import cv2
import numpy as np

In [2]:
net = cv2.dnn.readNet('/Users/amariofausta/Documents/Kuliah/Semester 8/Bobot/yolov3custom_last.weights copy 5','/Users/amariofausta/Documents/Kuliah/Semester 8/Bobot/yolov3custom2.cfg')
with open("/Users/amariofausta/Documents/Kuliah/Semester 8/Dataset/dataset/classes.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

In [3]:
classes

['Bola', 'Pembatas']

In [4]:
layer_names = net.getLayerNames()

output_layers = [layer_names[i- 1] for i in net.getUnconnectedOutLayers()]

In [5]:
erno = net.getUnconnectedOutLayers()
for i in erno:
    print(i)

200
227
254


In [6]:
img = cv2.imread("/Users/amariofausta/Documents/Kuliah/Semester 8/Dataset/dataset/File 173.jpg")
height, width, channels = img.shape
blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)


In [7]:
img

array([[[ 75,  65,  55],
        [ 73,  63,  53],
        [ 70,  61,  51],
        ...,
        [155, 159, 134],
        [156, 160, 135],
        [156, 160, 135]],

       [[ 76,  66,  56],
        [ 74,  64,  54],
        [ 71,  62,  52],
        ...,
        [156, 160, 135],
        [156, 160, 135],
        [156, 160, 135]],

       [[ 74,  66,  53],
        [ 72,  64,  51],
        [ 70,  62,  49],
        ...,
        [157, 161, 136],
        [156, 160, 135],
        [156, 160, 135]],

       ...,

       [[ 90,  88, 148],
        [ 89,  87, 147],
        [ 88,  86, 146],
        ...,
        [104, 103, 167],
        [104, 103, 167],
        [104, 103, 167]],

       [[ 91,  89, 149],
        [ 90,  88, 148],
        [ 88,  86, 146],
        ...,
        [104, 103, 167],
        [104, 103, 167],
        [105, 104, 168]],

       [[ 92,  90, 150],
        [ 90,  88, 148],
        [ 88,  86, 146],
        ...,
        [105, 104, 168],
        [106, 105, 169],
        [107, 106, 170]]

In [8]:
net.setInput(blob)
outs = net.forward(output_layers)

class_ids = []
confidences = []
boxes = []
for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.5:
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)
            x = center_x - w / 2
            y = center_y - h / 2
            boxes.append([x, y, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)

indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

font = cv2.FONT_HERSHEY_PLAIN
colors = np.random.uniform(0, 255, size=(len(classes), 3))

for i in range(len(boxes)):
    if i in indexes:
        x, y, w, h = boxes[i]
        label = str(classes[class_ids[i]])
        color = (0,255,0)
        cv2.rectangle(img, (int(x), int(y)), (int(x + w), int(y + h)), color, 10)
        cv2.putText(img, label, (int(x), int(y) - 5), font, 10, color, 10)

cv2.imshow("Image", img)
cv2.waitKey(0)
cv2.destroyAllWindows()